# Scala Things You Should Know

# Packages

```scala
package mytools
class Tool1 { ... }
```
This name can be used when referencing code defined in this file.  
```scala
import mytools.Tool1
```
>Note: The package name  **should** match the directory hierarchy, this is not mandatory but failing to abide by this guideline can produce some unusual and difficult to diagnose problems. Package names by convention are lower case and do not contain separators like underscore.  This sometimes makes good descriptive names difficult.  One approach is too add a layer of hierachy ```package good.tools```.  Do your best.  Chisel itself plays some games with the package names that do not conform to these rules.

# A Simple Class Example
An example of creating a Scala class might be

In [ ]:
class WrapCounter(counterBits: Int) {
  val max: Long = (1 << counterBits) - 1
  var counter = 0L
  def inc(): Long = {
    counter = counter + 1
    if(counter > max) counter = 0
    counter
  }
  println(s"counter created with max value $max")
}

What is here:
* ```class WrapCounter```: This is the definition of **WrapCounter**
* ```(counterBits: Int)```: Creating MyCounter requires an integer argument, nicely named to suggest it is the bit width of the counter
* braces ({}) delimit a block of code. Most classes use a code block to define variables and constants and methods (functions)
* ```val max: Long =``` the class contains a member variable **max**, declared as ```Long``` and initialized as the class is created
* ```(1 << counterBits) - 1``` computes the maximum value that can be contained in **counterBits** bits.  Since **max** was created with _val_ it cannot be changed.
* a variable **counter** is created and initialized to **0L**, the **L** says that 0 is a long value and from this **counter** is inferred to be Long.
* **max** and **counter** are commonly called _member variables_ of the class
* a class method **inc** is defined which takes no arguments and returns a **Long** value
* the body of the method **inc** is a code block that:
  * ```counter = counter + 1``` increments **counter**
  * ```if(counter > max) counter = 0``` tests if it is greater than the **max** value and sets it back to zero if it is
  * ```counter``` the last line of the code block is important
    * any value expressed at the last line of a code block is considered to be the return value of that code block, that return value can be used or ignore by the programmer
    * this applies quite generally, for example since an if statement or and if then else statement defines its true and false clauses with code blocks the if itself can return a value
    * for example ```val result = if( 10 * 10 > 90) { "greater" } else { "lesser" }``` would created a variable with the value "greater"
  * so in this case the function **inc** returns the value of **counter**
* ```println(s"counter created with max value $max")``` this prints a string to the standard out.  Because the **println** is directly in the defining code block it is part of the classes initialization code and is run, i.e. prints out the string, every time an instance of this class is created.
* The string printed in this case is an _interpolated_ string
  * the leading **s** in front of the first double quote identifies this as an interpolated
  * an interpolated string is processed at run time  
  * the **$max** is replaced with the value of max
  * if the **$** is followed by a code block arbitrary scala can be in that code block
    * for example **${max + max}**
    * the return value of this code block will be inserted  in place of ${...}
    * if the return value is not a string it will be converted to one, virtually every class or type in scala has implicit conversion to a string)
  * it should be noted that classes that print something every time an instance is created is darned annoying and should avoided


# Creating an instance of a class
Let's use our example above to create a class.  Scala instances are created via the built-in magic keyword **new**

In [ ]:
val x = new WrapCounter(4)

Now often in scala code one sees, instances being created without the keyword new, for example ```val y = WrapCounter(6)```
This occurs often enough to merit special attention.  I it described below under [Companion Objects](#Companion-Objects)


# Code Blocks
Code blocks are delimited by braces.  A block can contain zero or more lines of scala code. The last line of scala code becomes the return value (which may be ignored) of the code block.  A code block with no lines would return and special null-like object called Unit. Code blocks are used throughout scala, they are the bodies of class definitions, they form function and method definitions, they are the clauses of if statements, they are the bodies of for and many other scala operators.

### Parameterized Code Blocks
Code blocks can take parameters.  In the case of class and method definitions these parameters look fairly like most conventional programming languages.  In the example below ```c``` and ```s``` are parameters of the code block.


In [ ]:
def add1(c: Int): Int = {
  c + 1
}
class RepeatString(s: String) {
  val repeatedString = s + s
}

**IMPORTANT**: There is another way in which code blocks may be parameterized, it is visible all over the place in a scala program and I found it to be one of the scala constructs that took me a while to get used to.  Here are some examples

In [ ]:
val intList = List(1, 2, 3)
val stringList = intList.map { i =>
  i.toString
}

The code block is begin passed to a method map of the class List.  The map method requires that it's code block have a single parameter.  The code block is called for each member of list, the code block returns that parameter converted to a String. Scala is almost excessively accepting of variations of this syntax.  You might see this written in many different ways. Assuming intList is defined as above, all the following will return a list of string versions of the original integer listµ

In [ ]:
intList map ( s => s.toString )
intList.map( s => s.toString )
intList.map { s: Int => s.toString }
intList.map { (s: Int) => s.toString }

// Even more exotically one can create a variable which points to a code block

val intToString = { c: Int =>
  c + 1
}
intList.map(intToString)

// or even still more exotically periods and parens can be dropped entirely. 
intList map intToString

The goal again here is simply to help you recognize the different notational types when you encounter them.  As you use Scala these will seem more comfortable and familiar.  Authors tend to gravitate to particular styles and there are also individual syntactical situations in which one notation will seem more natural. One liners tend to use the more concise forms, complex blocks usually have a more narrative appearance.

# Named parameters
When a method is defined in scala, for example
```scala
def myMethod(count: Int, wrap: Boolean, wrapValue: Int = 24): Unit = { ... }
```
When calling the method, you will often see the parameter names along with the passed in values
```scala
myMethod(count = 10, wrap = false, wrapValue = 23)
```
For frequently called methods, the parameter ordering may be obvious but for less common methods and, in particular, boolean arguments, including the names with calls can make your code a lot more readable.  Using named parameters can allow you to re-arrange arguments, and in combination with parameters that have a default value, can make it so the caller only has to pass (by name) the specific arguments that do not use the default value.  Parameters to class definitions also used this named argument scheme (they are actually just the parameters to the constructor method for the class).

# String interpolation
There are lot of ways of building strings with dynamic content.  The simplest is simply to use the add operator.

In [ ]:
val (dogsName, destination) = ("Rufus", "Store")
println("my dog " + dogsName + " went to the " + destination)

But there are a lot of more alternative (possibly more elegant ways) to do the same thing.  The first is the **s** string interpolator.  When a string begins with **s"** any occurrences of a dollar sign followed by a scala variable, or a dollar sign followed by a code block, will have that replaced by the string value of the variable or block.  For our previous example we could make this 

In [ ]:
println(s"my dog $dogsName went to the $destination")


As an example of using a code block, let's assume we have a list and we'd like to print this out with each animal capitalized. 


In [ ]:
val aList = List("dog", "cat", "fox")
println(s"Animal list ${aList.map(s => s.capitalize).mkString(" -- ")}")


We used the ${...} to execute some scala code on that list.  map converts the list to a new list in which each word has been capitalized, then that new list is changed into a string by taking each element and joining it to the others with the string " -- " between each element.

Another less commonly used interpolator is the **f** which allows printf style formatting specifiers to be included at each interpolation point.

In [ ]:
case class Person(name: String, age: Int)

val (lineNumber, person) = (22, Person("blink", 77))
println(f"$lineNumber%6d ${person.name}%-40s ${person.age}%3d")


Can be used to make some columns that line up nicely.  Scala does have printf too if you really need it.  But be a little careful with that Chisel provides it's own version of printf specifically for debug printing inside an executing simulation.
